<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="900px"/> 

<a id="title_ID"></a>
# MIRI IMA Batch Processing Notebook #

**Author**: David Law, AURA Associate Astronomer, MIRI branch
<br>
**Last Updated**: September 12, 2022
<br>
**Pipeline Version**: 1.7.0+ (unreleased)

The purpose of this notebook is to provide a framework for bare-bones batch processing of MIRI MRS simultaneous imaging data.  Implementation is largely based on the JWebbinar MRS notebooks.  For more extensive options, astrometric tweaks, etc. see notebooks at https://github.com/STScI-MIRI/Imaging_ExampleNB

<hr style="border:1px solid gray"> </hr>

1.<font color='white'>-</font>Configuration <a class="anchor" id="intro"></a>
------------------

In [45]:
# Set parameters to be changed here
import sys,os

# CRDS context
#%env CRDS_CONTEXT jwst_0771.pmap

# input_dir should point to where the uncalibrated FITS files are
input_dir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/'

# output_dir should point to where you want the output results to go
output_dir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/'

# Reference functions to set step skips and ref file overrides
sys.path.append(os.path.join(os.environ.get('MIRIBOX'),'CAP/'))
import mrs_config as mconfig

dodet1=True
doimage2=True
doimage3=True

<hr style="border:1px solid gray"> </hr>

2.<font color='white'>-</font>Imports and setup <a class="anchor" id="intro"></a>
------------------

In [34]:
# Now let's use the entire available screen width for the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

2022-09-12 13:02:39,692 - stpipe - WARNING - /var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_43317/1813087124.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML



In [35]:
# Basic system utilities for interacting with files
import glob
import time
import shutil
import warnings
import zipfile
import urllib.request

# Astropy utilities for opening FITS and ASCII files
from astropy.io import fits
from astropy.io import ascii
from astropy.utils.data import download_file
# Astropy utilities for making plots
from astropy.visualization import (LinearStretch, LogStretch, ImageNormalize, ZScaleInterval)

# Numpy for doing calculations
import numpy as np

# Matplotlib for making plots
import matplotlib.pyplot as plt
from matplotlib import rc

In [36]:
# Import the base JWST package
import jwst

In [37]:
# JWST pipelines (encompassing many steps)
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline

# JWST pipeline utilities
from jwst import datamodels # JWST datamodels
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

from stcal import dqflags # Utilities for working with the data quality (DQ) arrays

import stcal.ramp_fitting.utils as utils # Utilities for handling multiprocessing

In [38]:
import miricoord.mrs.mrs_pipetools as mpt

In [50]:
# Specify output subdirectories to keep data products organized
det1_dir = os.path.join(output_dir, 'stage1/') # Detector1 pipeline outputs will go here
image2_dir = os.path.join(output_dir, 'stage2/') # Image2 pipeline outputs will go here
image3_dir = os.path.join(output_dir, 'stage3/') # Image3 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if not os.path.exists(det1_dir):
    os.makedirs(det1_dir)
if not os.path.exists(image2_dir):
    os.makedirs(image2_dir)
if not os.path.exists(image3_dir):
    os.makedirs(image3_dir)

In [9]:
# Start a timer to keep track of runtime
time0 = time.perf_counter()

<hr style="border:1px solid gray"> </hr>

3.<font color='white'>-</font>Detector1 Pipeline <a class="anchor" id="det1"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our simulated data through the Detector1 pipeline to create Lvl2a data products (i.e., uncalibrated slope images).
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html
</div>

In [10]:
# First we'll define a function that will call the detector1 pipeline with our desired set of parameters
# We won't enumerate the individual steps
def rundet1(filename):
    print(filename)
    det1 = Detector1Pipeline() # Instantiate the pipeline
    det1.output_dir = det1_dir # Specify where the output should go
    
        
    det1.save_results = True # Save the final resulting _rate.fits files
    det1(filename) # Run the pipeline on an input list of files

In [11]:
# Now let's look for input files of the form *uncal.fits
sstring = input_dir + 'jw*mirimage_uncal.fits'
lvl1bfiles = sorted(glob.glob(sstring))
print('Found ' + str(len(lvl1bfiles)) + ' input files to process')

Found 6 input files to process


In [12]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1:
    for file in lvl1bfiles:
        rundet1(file)
else:
    print('Skipping Detector1 processing')

2022-09-12 12:42:52,764 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-09-12 12:42:52,765 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-09-12 12:42:52,766 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-09-12 12:42:52,768 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-09-12 12:42:52,770 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-09-12 12:42:52,772 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-09-12 12:42:52,775 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-09-12 12:42:52,777 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-09-12 12:42:52,779 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-09-12 12:42:52,780 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-09-12 1

data/Obs04/uncal/jw02732004001_02101_00001_mirimage_uncal.fits


2022-09-12 12:42:55,009 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02101_00001_mirimage_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-09-12 12:42:56,245 - CRDS - INFO -  Fetching  /Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_mask_0033.fits    4.3 M bytes  (1 / 3 files) (0 / 17.0 M bytes)
2022-09-12 12:42:59,158 - CRDS - INFO -  Fetching  /Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_readnoise_0085.fits    4.2 M bytes  (2 / 3 files) (4.3 M / 17.0 M bytes)
2022-09-12 12:43:01,685 - CRDS - INFO -  Fetching  /Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_saturation_0034.fits    8.5 M bytes  (3 / 3 files) (8.5 M / 17.0 M bytes)
2022-09-12 12:43:05,155 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_dark_0082.fits

2022-09-12 12:43:10,831 - stpipe.Detector1Pipeline.reset - INFO - Using RESET reference file /Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_reset_0070.fits
2022-09-12 12:43:11,441 - stpipe.Detector1Pipeline.reset - INFO - Step reset done
2022-09-12 12:43:11,550 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(5, 10, 1024, 1032) from jw02732004001_02101_00001_mirimage_uncal.fits>,).
2022-09-12 12:43:11,552 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': 'data/Obs04/uncal'}
2022-09-12 12:43:11,581 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_linearity_0032.fits
2022-09-12 12:43:12,41

2022-09-12 12:43:45,389 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 10 pixels with at least one CR from four groups.
2022-09-12 12:43:45,390 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 13 pixels with at least one CR from three groups.
2022-09-12 12:43:50,691 - stpipe.Detector1Pipeline.jump - INFO - Working on integration 5:
2022-09-12 12:43:51,595 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 20655 pixels with at least one CR from five or more groups.
2022-09-12 12:43:51,596 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 10 pixels with at least one CR from four groups.
2022-09-12 12:43:51,596 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 11 pixels with at least one CR from three groups.
2022-09-12 12:43:57,525 - stpipe.Detector1Pipeline.jump - INFO - Total elapsed time = 32.2948 sec
2022-09-12 12:43:57,534 - stpipe.Detec

...

2022-09-12 12:50:40,349 - stpipe.Detector1Pipeline.jump - INFO - Working on integration 5:
2022-09-12 12:50:41,294 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 19277 pixels with at least one CR from five or more groups.
2022-09-12 12:50:41,295 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 17 pixels with at least one CR from four groups.
2022-09-12 12:50:41,296 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 6 pixels with at least one CR from three groups.
2022-09-12 12:50:46,834 - stpipe.Detector1Pipeline.jump - INFO - Total elapsed time = 33.1504 sec
2022-09-12 12:50:46,844 - stpipe.Detector1Pipeline.jump - INFO - The execution time in seconds: 33.589733
2022-09-12 12:50:46,851 - stpipe.Detector1Pipeline.jump - INFO - Step jump done
2022-09-12 12:50:46,985 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit running with args (<RampModel(5, 10, 1024, 1032) from jw02732004001_02105_0

data/Obs04/uncal/jw02732004001_02105_00002_mirimage_uncal.fits


2022-09-12 12:51:34,434 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02105_00002_mirimage_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-09-12 12:51:34,439 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_dark_0082.fits'.
2022-09-12 12:51:34,440 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_gain_0008.fits'.
2022-09-12 12:51:34,441 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is 'N/A'.
2022-09-12 12:51:34,442 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_linearity_0032.fits'.
2022-09-12 12:51:34,443 - stpipe.Detector1Pipeline - INFO - Prefetch for MA

2022-09-12 12:51:40,578 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': 'data/Obs04/uncal'}
2022-09-12 12:51:40,608 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_linearity_0032.fits
2022-09-12 12:51:41,464 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-09-12 12:51:41,595 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(5, 10, 1024, 1032) from jw02732004001_02105_00002_mirimage_uncal.fits>,).
2022-09-12 12:51:41,597 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fit

2022-09-12 12:52:23,407 - stpipe.Detector1Pipeline.jump - INFO - Working on integration 5:
2022-09-12 12:52:24,279 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 19740 pixels with at least one CR from five or more groups.
2022-09-12 12:52:24,280 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 14 pixels with at least one CR from four groups.
2022-09-12 12:52:24,280 - stpipe.Detector1Pipeline.jump - INFO - From highest outlier, two-point found 5 pixels with at least one CR from three groups.
2022-09-12 12:52:30,079 - stpipe.Detector1Pipeline.jump - INFO - Total elapsed time = 35.1886 sec
2022-09-12 12:52:30,089 - stpipe.Detector1Pipeline.jump - INFO - The execution time in seconds: 35.626267
2022-09-12 12:52:30,097 - stpipe.Detector1Pipeline.jump - INFO - Step jump done
2022-09-12 12:52:30,222 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit running with args (<RampModel(5, 10, 1024, 1032) from jw02732004001_02105_0

In [16]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 703.4723 seconds


<hr style="border:1px solid gray"> </hr>

4.<font color='white'>-</font>Image2Pipeline <a class="anchor" id="image2"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our data through the Image2 pipeline in order to produce Lvl2b data products (i.e., calibrated slope images).  
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html
</div>

In [17]:
# Define a function that will call the image2 pipeline with our desired set of parameters
# We'll list the individual steps just to make it clear what's running
def runimage2(filename):
    image2 = Image2Pipeline()
    image2.output_dir = image2_dir
    
    image2.save_results = True
    image2(filename)

In [18]:
# Look for input uncalibrated slope files from the Detector1 pipeline
sstring = det1_dir + 'jw*mirimage*rate.fits'
ratefiles = sorted(glob.glob(sstring))
ratefiles=np.array(ratefiles)
print('Found ' + str(len(ratefiles)) + ' input files to process')

Found 6 input files to process


In [19]:
if doimage2:
    for file in ratefiles:
        runimage2(file)
else:
    print('Skipping Image2 processing')

2022-09-12 12:54:32,888 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2022-09-12 12:54:32,891 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-09-12 12:54:32,895 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-09-12 12:54:32,897 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-09-12 12:54:32,900 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2022-09-12 12:54:32,903 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2022-09-12 12:54:33,016 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('data/Obs04/stage1/jw02732004001_02101_00001_mirimage_rate.fits',).
2022-09-12 12:54:33,022 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': 'data/Obs04/stage2/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index

2022-09-12 12:54:44,181 - stpipe.Image2Pipeline.assign_wcs - INFO - Update S_REGION to POLYGON ICRS  339.028045507 34.021326504 338.994058805 34.007725009 339.010491608 33.979414698 339.044676624 33.992772186
2022-09-12 12:54:44,182 - stpipe.Image2Pipeline.assign_wcs - INFO - assign_wcs updated S_REGION to POLYGON ICRS  339.028045507 34.021326504 338.994058805 34.007725009 339.010491608 33.979414698 339.044676624 33.992772186
2022-09-12 12:54:44,182 - stpipe.Image2Pipeline.assign_wcs - INFO - COMPLETED assign_wcs
2022-09-12 12:54:44,291 - stpipe.Image2Pipeline.assign_wcs - INFO - Step assign_wcs done
2022-09-12 12:54:44,402 - stpipe.Image2Pipeline.flat_field - INFO - Step flat_field running with args (<ImageModel(1024, 1032) from jw02732004001_02101_00001_mirimage_rate.fits>,).
2022-09-12 12:54:44,403 - stpipe.Image2Pipeline.flat_field - INFO - Step flat_field parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use

2022-09-12 12:54:48,899 - stpipe.Image2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02101_00002_mirimage_rate.fits' reftypes = ['area', 'camera', 'collimator', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'photom', 'regions', 'sflat', 'specwcs', 'wavelengthrange', 'wfssbkg']
2022-09-12 12:54:48,905 - stpipe.Image2Pipeline - INFO - Prefetch for AREA reference file is '/Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_area_0004.fits'.
2022-09-12 12:54:48,906 - stpipe.Image2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2022-09-12 12:54:48,907 - stpipe.Image2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2022-09-12 12:54:48,908 - stpipe.Image2Pipeline - INFO - Prefetch for DFLAT reference file is 'N/A'.
2022-09-12 12:54:48,908 - stpipe.Image2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2022-09-12 12:54

2022-09-12 12:54:50,372 - stpipe.Image2Pipeline.photom - INFO - PHOTMJSR value: 0.304428
2022-09-12 12:54:50,394 - stpipe.Image2Pipeline.photom - INFO - Step photom done
2022-09-12 12:54:50,508 - stpipe.Image2Pipeline.resample - INFO - Step resample running with args (<ImageModel(1024, 1032) from jw02732004001_02101_00002_mirimage_rate.fits>,).
2022-09-12 12:54:50,510 - stpipe.Image2Pipeline.resample - INFO - Step resample parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'i2d', 'search_output_file': True, 'input_dir': 'data/Obs04/stage1', 'pixfrac': 1.0, 'kernel': 'square', 'fillval': 'INDEF', 'weight_type': 'ivm', 'output_shape': None, 'crpix': None, 'crval': None, 'rotation': None, 'pixel_scale_ratio': 1.0, 'pixel_scale': None, 'single': False, 'blendheaders': True, 'allowed_memory': None, 'in_memory': True}
2022-09-12 

2022-09-12 12:56:13,401 - stpipe.Image2Pipeline - INFO - Prefetch for FLAT reference file is '/Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_flat_0786.fits'.
2022-09-12 12:56:13,402 - stpipe.Image2Pipeline - INFO - Prefetch for FORE reference file is 'N/A'.
2022-09-12 12:56:13,403 - stpipe.Image2Pipeline - INFO - Prefetch for FPA reference file is 'N/A'.
2022-09-12 12:56:13,403 - stpipe.Image2Pipeline - INFO - Prefetch for IFUFORE reference file is 'N/A'.
2022-09-12 12:56:13,404 - stpipe.Image2Pipeline - INFO - Prefetch for IFUPOST reference file is 'N/A'.
2022-09-12 12:56:13,405 - stpipe.Image2Pipeline - INFO - Prefetch for IFUSLICER reference file is 'N/A'.
2022-09-12 12:56:13,405 - stpipe.Image2Pipeline - INFO - Prefetch for MSA reference file is 'N/A'.
2022-09-12 12:56:13,406 - stpipe.Image2Pipeline - INFO - Prefetch for OTE reference file is 'N/A'.
2022-09-12 12:56:13,407 - stpipe.Image2Pipeline - INFO - Prefetch for PHOTOM reference file is '/Users/dlaw/crds_cache/referenc

...

2022-09-12 12:56:18,839 - stpipe.Image2Pipeline - INFO - Working on input data/Obs04/stage1/jw02732004001_02103_00002_mirimage_rate.fits ...
2022-09-12 12:56:19,052 - stpipe.Image2Pipeline.assign_wcs - INFO - Step assign_wcs running with args (<ImageModel(1024, 1032) from jw02732004001_02103_00002_mirimage_rate.fits>,).
2022-09-12 12:56:19,054 - stpipe.Image2Pipeline.assign_wcs - INFO - Step assign_wcs parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': 'data/Obs04/stage1', 'sip_approx': True, 'sip_max_pix_error': 0.25, 'sip_degree': None, 'sip_max_inv_pix_error': 0.25, 'sip_inv_degree': None, 'sip_npoints': 32, 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2022-09-12 12:56:19,489 - stpipe.Image2Pipeline.assign_wcs - INFO - Created a MIRI mir_image pipeline with references {'distor

2022-09-12 12:56:24,226 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-09-12 12:56:24,228 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-09-12 12:56:24,230 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-09-12 12:56:24,232 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2022-09-12 12:56:24,235 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2022-09-12 12:56:24,363 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('data/Obs04/stage1/jw02732004001_02105_00001_mirimage_rate.fits',).
2022-09-12 12:56:24,373 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': 'data/Obs04/stage2/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 

2022-09-12 12:58:29,572 - stpipe.Image2Pipeline.assign_wcs - INFO - assign_wcs updated S_REGION to POLYGON ICRS  339.028066883 34.021329819 338.994080500 34.007728654 339.010513542 33.979418184 339.044697894 33.992775579
2022-09-12 12:58:29,573 - stpipe.Image2Pipeline.assign_wcs - INFO - COMPLETED assign_wcs
2022-09-12 12:58:29,669 - stpipe.Image2Pipeline.assign_wcs - INFO - Step assign_wcs done
2022-09-12 12:58:29,772 - stpipe.Image2Pipeline.flat_field - INFO - Step flat_field running with args (<ImageModel(1024, 1032) from jw02732004001_02105_00001_mirimage_rate.fits>,).
2022-09-12 12:58:29,773 - stpipe.Image2Pipeline.flat_field - INFO - Step flat_field parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': 'data/Obs04/stage1', 'save_interpolated_flat': False, 'user_supplied_fla

2022-09-12 12:58:34,405 - stpipe.Image2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02105_00002_mirimage_rate.fits' reftypes = ['area', 'camera', 'collimator', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'photom', 'regions', 'sflat', 'specwcs', 'wavelengthrange', 'wfssbkg']
2022-09-12 12:58:34,410 - stpipe.Image2Pipeline - INFO - Prefetch for AREA reference file is '/Users/dlaw/crds_cache/references/jwst/miri/jwst_miri_area_0004.fits'.
2022-09-12 12:58:34,411 - stpipe.Image2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2022-09-12 12:58:34,411 - stpipe.Image2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2022-09-12 12:58:34,412 - stpipe.Image2Pipeline - INFO - Prefetch for DFLAT reference file is 'N/A'.
2022-09-12 12:58:34,413 - stpipe.Image2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2022-09-12 12:58

2022-09-12 12:58:35,923 - stpipe.Image2Pipeline.photom - INFO - PHOTMJSR value: 1.18843
2022-09-12 12:58:35,950 - stpipe.Image2Pipeline.photom - INFO - Step photom done
2022-09-12 12:58:36,051 - stpipe.Image2Pipeline.resample - INFO - Step resample running with args (<ImageModel(1024, 1032) from jw02732004001_02105_00002_mirimage_rate.fits>,).
2022-09-12 12:58:36,053 - stpipe.Image2Pipeline.resample - INFO - Step resample parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'i2d', 'search_output_file': True, 'input_dir': 'data/Obs04/stage1', 'pixfrac': 1.0, 'kernel': 'square', 'fillval': 'INDEF', 'weight_type': 'ivm', 'output_shape': None, 'crpix': None, 'crval': None, 'rotation': None, 'pixel_scale_ratio': 1.0, 'pixel_scale': None, 'single': False, 'blendheaders': True, 'allowed_memory': None, 'in_memory': True}
2022-09-12 1

In [20]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 953.1897 seconds


<hr style="border:1px solid gray"> </hr>

5.<font color='white'>-</font>Image3Pipeline <a class="anchor" id="image3"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our data through the Image3 pipeline in order to produce drizzle-combined data products.  
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html
</div>

In [54]:
# Define a useful function to write out a Lvl3 association file from an input list
def writel3asn(scifiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMS_Level3_Base, product_name=prodname)
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

In [55]:
# Define a function that will call the image3 pipeline with our desired set of parameters
# We'll list the individual steps just to make it clear what's running
def runimage3(filename):
    image3 = Image3Pipeline()
    image3.output_dir = image3_dir
    
    image3.save_results = True
    image3(filename)

In [56]:
# Find and sort all of the input files

# Science Files need the cal.fits files
sstring = image2_dir + 'jw*mirimage*cal.fits'
calfiles = np.array(sorted(glob.glob(sstring)))

print('Found ' + str(len(calfiles)) + ' science files to process')

Found 6 science files to process


In [57]:
# Make an association file that includes all of the different exposures
asnfile=os.path.join(output_dir, 'l3imaasn.json')
if doimage3:
    writel3asn(calfiles, asnfile, 'Level3')

In [58]:
if doimage3:
    runimage3(asnfile)
else:
    print('Skipping Image3 processing')

2022-09-12 13:43:46,968 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2022-09-12 13:43:46,971 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2022-09-12 13:43:46,974 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2022-09-12 13:43:46,977 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2022-09-12 13:43:46,981 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2022-09-12 13:43:46,984 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2022-09-12 13:43:46,987 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.
2022-09-12 13:43:47,168 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline running with args ('/Users/dlaw/Box/dlaw/CambridgeWorkshop/data/Obs04/l3imaasn.json',).
2022-09-12 13:43:47,178 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [],

2022-09-12 13:43:52,357 - stpipe.Image3Pipeline.tweakreg - INFO -      jw02732004001_02103_00001_mirimage_cal
2022-09-12 13:43:52,392 - stpipe.Image3Pipeline.tweakreg - INFO - * Images in GROUP 'jw02732004001_02103_00002_mirimage_cal':
2022-09-12 13:43:52,393 - stpipe.Image3Pipeline.tweakreg - INFO -      jw02732004001_02103_00002_mirimage_cal
2022-09-12 13:43:52,428 - stpipe.Image3Pipeline.tweakreg - INFO - * Images in GROUP 'jw02732004001_02105_00001_mirimage_cal':
2022-09-12 13:43:52,429 - stpipe.Image3Pipeline.tweakreg - INFO -      jw02732004001_02105_00001_mirimage_cal
2022-09-12 13:43:52,465 - stpipe.Image3Pipeline.tweakreg - INFO - * Images in GROUP 'jw02732004001_02105_00002_mirimage_cal':
2022-09-12 13:43:52,465 - stpipe.Image3Pipeline.tweakreg - INFO -      jw02732004001_02105_00002_mirimage_cal
2022-09-12 13:43:52,466 - stpipe.Image3Pipeline.tweakreg - INFO - 
2022-09-12 13:43:52,467 - stpipe.Image3Pipeline.tweakreg - INFO -  
2022-09-12 13:43:52,468 - stpipe.Image3Pipeline

2022-09-12 13:44:04,623 - stpipe.Image3Pipeline.skymatch - INFO -    *  Image ID=jw02732004001_02101_00002_mirimage_cal.fits. Sky background: 0
2022-09-12 13:44:04,624 - stpipe.Image3Pipeline.skymatch - INFO -    *  Image ID=jw02732004001_02103_00001_mirimage_cal.fits. Sky background: 9.50842
2022-09-12 13:44:04,625 - stpipe.Image3Pipeline.skymatch - INFO -    *  Image ID=jw02732004001_02103_00002_mirimage_cal.fits. Sky background: 9.53502
2022-09-12 13:44:04,626 - stpipe.Image3Pipeline.skymatch - INFO -    *  Image ID=jw02732004001_02105_00001_mirimage_cal.fits. Sky background: 17.2704
2022-09-12 13:44:04,627 - stpipe.Image3Pipeline.skymatch - INFO -    *  Image ID=jw02732004001_02105_00002_mirimage_cal.fits. Sky background: 17.0807
2022-09-12 13:44:04,627 - stpipe.Image3Pipeline.skymatch - INFO -  
2022-09-12 13:44:04,628 - stpipe.Image3Pipeline.skymatch - INFO - ***** jwst.skymatch.skymatch.match() ended on 2022-09-12 13:44:04.627866
2022-09-12 13:44:04,629 - stpipe.Image3Pipeline.s

2022-09-12 13:44:29,124 - stpipe.Image3Pipeline.outlier_detection - INFO - Saved model in /Users/dlaw/Box/dlaw/CambridgeWorkshop/data/Obs04/stage3/jw02732004001_02105_00002_mirimage_a3001_crf.fits
2022-09-12 13:44:29,124 - stpipe.Image3Pipeline.outlier_detection - INFO - Step outlier_detection done
2022-09-12 13:44:29,311 - stpipe.Image3Pipeline.resample - INFO - Step resample running with args (<ModelContainer>,).
2022-09-12 13:44:29,314 - stpipe.Image3Pipeline.resample - INFO - Step resample parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'i2d', 'search_output_file': True, 'input_dir': '/Users/dlaw/Box/dlaw/CambridgeWorkshop/data/Obs04', 'pixfrac': 1.0, 'kernel': 'square', 'fillval': 'INDEF', 'weight_type': 'ivm', 'output_shape': None, 'crpix': None, 'crval': None, 'rotation': None, 'pixel_scale_ratio': 1.0, 'pixel_sca

In [ ]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://www.stsci.edu/~dlaw/stsci_logo.png" alt="stsci_logo" width="200px"/> 